Global setup



In [8]:
n=1
k=4
q=23 #AA
eta=2

Bob generates a key


In [9]:
#S = random_matrix(ZZ, 4, 4, distribution='gaussian', x=2)
S = matrix(ZZ, [[ 3,  0,  2, -3],
                [ 0,  3, -2, -1],
                [-1,  0,  1, -1],
                [ 1,  1, -2,  2]])
S

[ 3  0  2 -3]
[ 0  3 -2 -1]
[-1  0  1 -1]
[ 1  1 -2  2]

$S$ is the private key


In [10]:
#T = matrix(((A*S + E) % q))
T = matrix(ZZ, [[ 8 , 14 , 15 , 9],
[7 , 16 , 21 , 12],
[21 , 0 , 16 , 1],
[2 , 9 , 5 , 11]])
T

[ 8 14 15  9]
[ 7 16 21 12]
[21  0 16  1]
[ 2  9  5 11]

$(A,T)$ is the public key


Alice computes 



In [4]:
r = matrix([[0, 1, -1, -1]]); r

[ 0  1 -1 -1]

In [5]:
e1 = matrix([[1, 0, 0, 0]]); e1

[1 0 0 0]

In [13]:
u = matrix([[13, 4, 5, 22]]); u


[13  4  5 22]

**Idea for lattice attack recovering $A$ from $s$ and $u$**

(Assuming $r$ and $e_1$ are correct)

Treating matrices by rows:

$$\mathbf{a_i}S + \mathbf{e_i} - \mathbf{t_i}\equiv \mathbf{0} \pmod{q}$$
$$\mathbf{r}\mathbf{a_i}^T + \mathbf{e_{1i}} - \mathbf{u_i}\equiv \mathbf{0} \pmod{q}$$

equivalently:

$$\mathbf{a_i}S + \mathbf{e_i} - \mathbf{t_i}\equiv \mathbf{0} \pmod{q}$$
$$\mathbf{a_i}\mathbf{r}^T + {e_{1i}} - {u_i}\equiv \mathbf{0} \pmod{q}$$

or in block row form, $$(\mathbf{a_i} | \mathbf{e} | 1 | \mathbf{s'})\begin{pmatrix} S \\ - \\ I \\ - \\ -\mathbf{t_i} \\ - \\ qI \end{pmatrix} =  \mathbf{0} $$

and $$(\mathbf{a_i} |  | 1 | {s_1'})\begin{pmatrix} \mathbf{r}^T \\ - \\ \\ - \\ e_{i1}-{u_i} \\ - \\ q \end{pmatrix} = \mathbf{0} $$

together:  $$(\mathbf{a_i} | \mathbf{e} |  |1 | \mathbf{s'} | {s_1'})\begin{pmatrix} S & \mathbf{r}^T\\ - & - \\ I & 0\\ - & - \\  & \\ - & - \\ -\mathbf{t_i} & {e_{1i}}-{u_i}\\ - & - \\ qI & 0 \\ - & - \\ 0 & q\end{pmatrix} =  \mathbf{0} $$

In [20]:
I4 = identity_matrix(4)
I4

[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]

In [23]:
Z4 = column_matrix([0,0,0,0]); Z4

[0]
[0]
[0]
[0]

In [17]:
t0 = matrix(T[0])
t0

[ 8 14 15  9]

In [32]:
u0 = matrix(1,1,u[0][0]); u0

[13]

In [45]:
e10 = matrix(1,1,e1[0][0]); e10

[1]

In [84]:
M = block_matrix([[S, r.transpose()], [I4, Z4], [-t0, e10-u0], [q*I4, Z4], [0,q]])
M

[  3   0   2  -3|  0]
[  0   3  -2  -1|  1]
[ -1   0   1  -1| -1]
[  1   1  -2   2| -1]
[---------------+---]
[  1   0   0   0|  0]
[  0   1   0   0|  0]
[  0   0   1   0|  0]
[  0   0   0   1|  0]
[---------------+---]
[ -8 -14 -15  -9|-12]
[---------------+---]
[ 23   0   0   0|  0]
[  0  23   0   0|  0]
[  0   0  23   0|  0]
[  0   0   0  23|  0]
[---------------+---]
[  0   0   0   0| 23]

In [85]:
M.kernel()

Free module of degree 14 and rank 9 over Integer Ring
Echelon basis matrix:
[ 1  0  0  0 20  0 21  3  0 -1  0 -1  0  0]
[ 0  1  0  0 16  2  9 19  2  0  1  1  0  1]
[ 0  0  1  0  8 18 15  6 21  7 12 13  8 11]
[ 0  0  0  1  6 17 18  3 21  7 12 13  8 11]
[ 0  0  0  0 23  0  0  0  0 -1  0  0  0  0]
[ 0  0  0  0  0 23  0  0  0  0 -1  0  0  0]
[ 0  0  0  0  0  0 23  0  0  0  0 -1  0  0]
[ 0  0  0  0  0  0  0 23  0  0  0  0 -1  0]
[ 0  0  0  0  0  0  0  0 23  8 14 15  9 12]

In [86]:
M1 = block_matrix([[M, identity_matrix(14)]])
M1

[  3   0   2  -3   0|  1   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   3  -2  -1   1|  0   1   0   0   0   0   0   0   0   0   0   0   0   0]
[ -1   0   1  -1  -1|  0   0   1   0   0   0   0   0   0   0   0   0   0   0]
[  1   1  -2   2  -1|  0   0   0   1   0   0   0   0   0   0   0   0   0   0]
[  1   0   0   0   0|  0   0   0   0   1   0   0   0   0   0   0   0   0   0]
[  0   1   0   0   0|  0   0   0   0   0   1   0   0   0   0   0   0   0   0]
[  0   0   1   0   0|  0   0   0   0   0   0   1   0   0   0   0   0   0   0]
[  0   0   0   1   0|  0   0   0   0   0   0   0   1   0   0   0   0   0   0]
[ -8 -14 -15  -9 -12|  0   0   0   0   0   0   0   0   1   0   0   0   0   0]
[ 23   0   0   0   0|  0   0   0   0   0   0   0   0   0   1   0   0   0   0]
[  0  23   0   0   0|  0   0   0   0   0   0   0   0   0   0   1   0   0   0]
[  0   0  23   0   0|  0   0   0   0   0   0   0   0   0   0   0   1   0   0]
[  0   0   0  23   0|  0   0   0   0   0   0   0   0   0   0   0

In [87]:
H1 = M1.hermite_form()
H1 # The right side should be a transformation matrix (after some extra unnecessary but not harmful reduction)

[ 1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0]
[ 0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
[ 0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
[ 0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0]
[ 0  0  0  0  1  0  0  0  0  7 18 16  5 21  7 12 13  8 11]
[ 0  0  0  0  0  1  0  0  0 20  0 21  3  0 -1  0 -1  0  0]
[ 0  0  0  0  0  0  1  0  0 16  2  9 19  2  0  1  1  0  1]
[ 0  0  0  0  0  0  0  1  0  8 18 15  6 21  7 12 13  8 11]
[ 0  0  0  0  0  0  0  0  1  6 17 18  3 21  7 12 13  8 11]
[ 0  0  0  0  0  0  0  0  0 23  0  0  0  0 -1  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0 23  0  0  0  0 -1  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0 23  0  0  0  0 -1  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0 23  0  0  0  0 -1  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0 23  8 14 15  9 12]

In [89]:
N1 = H1.submatrix(5, 5)
N1 # A basis for the kernel

[ 1  0  0  0 20  0 21  3  0 -1  0 -1  0  0]
[ 0  1  0  0 16  2  9 19  2  0  1  1  0  1]
[ 0  0  1  0  8 18 15  6 21  7 12 13  8 11]
[ 0  0  0  1  6 17 18  3 21  7 12 13  8 11]
[ 0  0  0  0 23  0  0  0  0 -1  0  0  0  0]
[ 0  0  0  0  0 23  0  0  0  0 -1  0  0  0]
[ 0  0  0  0  0  0 23  0  0  0  0 -1  0  0]
[ 0  0  0  0  0  0  0 23  0  0  0  0 -1  0]
[ 0  0  0  0  0  0  0  0 23  8 14 15  9 12]

In [90]:
L = N1.LLL()
L

[ 0 -1 -1  0 -1  3 -1 -2  0  0  0  0  0  0]
[-1  0  0  0  3  0  2 -3  0  0  0  0  0  0]
[ 0  0 -1  1 -2 -1  3 -3  0  0  0  0  0  0]
[-4  1 -2  2  1  0  0  1  2  1  1  2  0  1]
[ 0 -1 -2  2  3 -4 -3 -2 -2 -1 -1 -1 -1 -1]
[ 4 -1 -4  3  4  0  0  1  0 -1  0  0  0  0]
[ 0 -2  4  4  1 -2 -1 -2  3  1  2  2  1  2]
[-2 -6  4  2  0  2  0  2 -1  0  0 -1 -1  0]
[ 3 -3 -2 -3  1 -1 -2  0  4  1  3  2  2  2]

In [101]:
L*M

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

Possibilities for $a_0$ must have 1 in 9th (1-based) position and small 5th through 8th positions

In [92]:
-L[7]

(2, 6, -4, -2, 0, -2, 0, -2, 1, 0, 0, 1, 1, 0)

In [93]:
a0 = matrix([2, 6, -4, -2])%q; a0

[ 2  6 19 21]

In [94]:
e0 = matrix([0, -2, 0, -2]); e0

[ 0 -2  0 -2]

In [105]:
(a0*S+e0)%q == t0

True

In [104]:
(a0*r.transpose()+e10)%q == u0

True

In [110]:
t1 = matrix(T[1])
u1 = matrix(1,1,u[0][1])
e11 = matrix(1,1,e1[0][1])
M = block_matrix([[S, r.transpose()], [I4, Z4], [-t1, e11-u1], [q*I4, Z4], [0,q]])
M1 = block_matrix([[M, identity_matrix(14)]])
H1 = M1.hermite_form()
N1 = H1.submatrix(5,5)
L = N1.LLL()
L

[ 0 -1 -1  0 -1  3 -1 -2  0  0  0  0  0  0]
[-1  0  0  0  3  0  2 -3  0  0  0  0  0  0]
[ 0  0 -1  1 -2 -1  3 -3  0  0  0  0  0  0]
[ 2  4  0  0  1  4  2 -1  1  0  0  1  1  0]
[ 5  1  2  3  0  1 -2  0 -1 -1 -1 -1  0  0]
[ 0 -2 -6  0  1 -1  0  4  1  0  1  1  0  0]
[-3  2  2  4  0 -3 -5 -2 -1  0 -1  0 -1  0]
[-1 -2  3  6 -2  2  1 -1  3  1  2  3  1  1]
[ 5  0  2 -6  0 -1 -3 -5  1  0  1  0  2  0]

In [111]:
-L[4]

(-5, -1, -2, -3, 0, -1, 2, 0, 1, 1, 1, 1, 0, 0)

In [112]:
a1 = matrix([-5, -1, -2, -3]); a1%q

[18 22 21 20]

In [113]:
E1 = matrix([0, -1, 2, 0]); E1

[ 0 -1  2  0]

In [115]:
(a1*S+E1)%q == t1

True

In [143]:
(a1*r.transpose()+e11)%q == u1

True

In [126]:
t2 = matrix(T[2])
u2 = matrix(1,1,u[0][2])
e12 = matrix(1,1,e1[0][2])
M = block_matrix([[S, r.transpose()], [I4, Z4], [-t2, e12-u2], [q*I4, Z4], [0,q]])
M1 = block_matrix([[M, identity_matrix(14)]])
H1 = M1.hermite_form()
N1 = H1.submatrix(5,5)
L = N1.LLL()
L

[ 0 -1 -1  0 -1  3 -1 -2  0  0  0  0  0  0]
[-1  0  0  0  3  0  2 -3  0  0  0  0  0  0]
[ 0  0  1 -1  2  1 -3  3  0  0  0  0  0  0]
[-1  1 -3 -1 -1 -2 -2 -2  1  1  0  1  0  0]
[-2 -1 -2  6  0 -3  0  1 -1 -1  0  0 -1  0]
[ 4 -1 -4  3  4  0  0  1  0 -1  0  0  0  0]
[-1 -1  5  2  0  1  1  0  3  3  0  2  0  1]
[ 1 -6 -2 -4 -1 -1  3  3  0  0  1 -1  0  0]
[-2  7 -2 -1  1  3  4  3  2  2 -1  2  0  0]

In [127]:
L*M

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

In [128]:
L[3]

(-1, 1, -3, -1, -1, -2, -2, -2, 1, 1, 0, 1, 0, 0)

In [129]:
a2 = matrix([-1, 1, -3, -1]); a2%q

[22  1 20 22]

In [130]:
e2 = matrix([-1, -2, -2, -2]); e2

[-1 -2 -2 -2]

In [134]:
(a2*S+e2)%q == matrix(T[2])

True

In [144]:
(a2*r.transpose()+e12)%q == u2

True

In [135]:
t3 = matrix(T[3])
u3 = matrix(1,1,u[0][3])
e13 = matrix(1,1,e1[0][3])
M = block_matrix([[S, r.transpose()], [I4, Z4], [-t3, e13-u3], [q*I4, Z4], [0,q]])
M1 = block_matrix([[M, identity_matrix(14)]])
H1 = M1.hermite_form()
N1 = H1.submatrix(5,5)
L = N1.LLL()
L

[ 0 -1 -1  0 -1  3 -1 -2  0  0  0  0  0  0]
[-1  0  0  0  3  0  2 -3  0  0  0  0  0  0]
[ 0  0 -1  1 -2 -1  3 -3  0  0  0  0  0  0]
[-1  2  0  0 -1 -1 -4  0 -2  0 -1  0 -1 -2]
[ 2  2  3  0 -1  3  2 -1  1  0  0  0  1  1]
[ 1 -2  1 -4  0  1  3 -1 -1  0  0 -1  0 -1]
[ 4 -1 -4  3  4  0  0  1  0 -1  0  0  0  0]
[-3  4 -1  2  0  5  4  3 -3  0 -2  0 -2 -3]
[ 3  7 -6 -6 -1 -2 -1 -3  4  0  1  1  3  3]

In [136]:
L*M

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

In [138]:
-L[3]-L[4]

(-1, -4, -3, 0, 2, -2, 2, 1, 1, 0, 1, 0, 0, 1)

In [140]:
a3 = matrix([-1, -4, -3, 0]); a3%q

[22 19 20  0]

In [141]:
e3 = matrix([2, -2, 2, 1]); e3

[ 2 -2  2  1]

In [142]:
(a3*S+e3)%q == matrix(T[3])

True

In [145]:
(a3*r.transpose()+e13)%q == u3

True

In [146]:
A = block_matrix(4, 1, [a0, a1, a2, a3])
A % q

[ 2  6 19 21]
[18 22 21 20]
[22  1 20 22]
[22 19 20  0]

In [147]:
E = block_matrix(4, 1, [e0, E1, e2, e3])
E

[ 0 -2  0 -2]
[-----------]
[ 0 -1  2  0]
[-----------]
[-1 -2 -2 -2]
[-----------]
[ 2 -2  2  1]

In [148]:
(A*S + E) %q 

[ 8 14 15  9]
[ 7 16 21 12]
[21  0 16  1]
[ 2  9  5 11]

Want $T = (AS+E) \bmod q$ 

and $u = (A^Tr+e_1) \bmod q$ 

$\Leftrightarrow u^T = (r^TA+e_1^T) \bmod q$

In [155]:
(A*S + E) %q == T

True

In [153]:
(A*r.transpose()+e1.transpose())%q # Not what I wanted!

[13]
[ 4]
[ 5]
[22]

In [154]:
u

[13  4  5 22]